In [1]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.base import AgentConfigName
from automata.config.openai_agent import OpenAIAutomataAgentConfigBuilder
from automata.agent.providers import OpenAIAutomataAgent
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.tools.factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

Loading modules with root path: /Users/ocolegrove/automata_fresh_2/automata/core/../.. and py path: /Users/ocolegrove/automata_fresh_2/automata/core/../../automata


In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["document-oracle","py-reader"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

Building dependencies for toolkits ['document-oracle', 'py-reader']...
Building py_reader...
Creating dependency py_reader
Building symbol_search...
Creating dependency symbol_search
Creating dependency symbol_graph
Creating dependency symbol_code_embedding_handler
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Successfully imported ClickHouse Connect C data optimizations
Using python library for writing JSON byte strings
loaded in 876 embeddings
loaded in 1 collections
collection with name automata already exists, returning existing collection
Creating dependency embedding_similarity_calculator
Building symbol_doc_embedding_handler...
Creating dependency symbol_doc_embedding_handler
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
loaded in 1428 embeddings
loaded in 1 collections
collection with name automata already exists, returning existing collection
Creating dependency py_context_handler
Cre

In [3]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [4]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-4")
    .with_max_iterations(3)
    .build()
)

Creating dependency symbol_rank
Creating dependency subgraph
Pre-computing bounding boxes for all rankable symbols
Error computing bounding box for scip-python python automata 85358531cb1d06a1b53846f8f365e186e986700b `automata.tools.builders.context_oracle`/ContextOracleOpenAIToolkitBuilder#: Module automata.tools.builders.context_oracle not found
Error computing bounding box for scip-python python automata 85358531cb1d06a1b53846f8f365e186e986700b `automata.tools.builders.context_oracle`/ContextOracleOpenAIToolkitBuilder#build_for_open_ai().: Module automata.tools.builders.context_oracle not found
Error computing bounding box for scip-python python automata 85358531cb1d06a1b53846f8f365e186e986700b `automata.tools.builders.context_oracle`/ContextOracleToolkitBuilder#: Module automata.tools.builders.context_oracle not found
Error computing bounding box for scip-python python automata 85358531cb1d06a1b53846f8f365e186e986700b `automata.tools.builders.context_oracle`/ContextOracleToolkitBui

In [5]:
import textwrap
instructions = textwrap.dedent('''
Provide a markdown python snippet, which can validly executed by `exec`. When ran, the snippet produces a valid instance of an openai agent config in the local variable `x`.
''')
agent = OpenAIAutomataAgent(instructions, config=agent_config)

In [6]:
# Run the agent
result = agent.run()

Function Call:
py-retriever-code

Arguments:
{
  "module_path": "automata.config.openai_agent",
  "node_path": "OpenAIAutomataAgentConfig"
}

 Function Call:
call_termination

Arguments:
{
  "result": "```python\nfrom automata.config.openai_agent import OpenAIAutomataAgentConfig\n\nx = OpenAIAutomataAgentConfig()\nx.setup()\n```"
}

 

In [7]:
# Print the result
print(f"Result:\n{result}")

Result:
```python
from automata.config.openai_agent import OpenAIAutomataAgentConfig

x = OpenAIAutomataAgentConfig()
x.setup()
```


In [8]:
result='```python\nfrom automata.config.openai_agent import OpenAIAutomataAgentConfig\n\nx = OpenAIAutomataAgentConfig()\nx.setup()```'
result='```python\n# We need to first import PyReader\nfrom automata.tools.builders.py_reader import PyReader\n\n# Next we create an instance of PyReader and assign it to the variable x\nx = PyReader()\n```'
cleaned_result = result.split('```python\n')[1].replace('```','')
print(cleaned_result)

# We need to first import PyReader
from automata.tools.builders.py_reader import PyReader

# Next we create an instance of PyReader and assign it to the variable x
x = PyReader()



In [9]:
exec(cleaned_result)

In [10]:
from automata.eval import (
    AgentEval,
    AgentEvalSetLoader,
    AgentEvaluationHarness,
    CodeWritingEval,
    OpenAIFunctionEval,
    CodeWritingAction
)
CodeWritingAction._extract_snippet(result)

'\n# We need to first import PyReader\nfrom automata.tools.builders.py_reader import PyReader\n\n# Next we create an instance of PyReader and assign it to the variable x\nx = PyReader()\n'